In [2]:
class State:
    def __init__(self, cannibal_left, missionary_left, boat, cannibal_right, missionary_right):
        self.cannibalLeft = cannibal_left
        self.missionaryLeft = missionary_left
        self.boat = boat
        self.cannibalRight = cannibal_right
        self.missionaryRight = missionary_right
        self.parent = None

    def is_goal(self):
        return self.cannibalLeft == 0 and self.missionaryLeft == 0

    def is_valid(self):
        if self.missionaryLeft >= 0 and self.missionaryRight >= 0 \
           and self.cannibalLeft >= 0 and self.cannibalRight >= 0 \
           and (self.missionaryLeft == 0 or self.missionaryLeft >= self.cannibalLeft) \
           and (self.missionaryRight == 0 or self.missionaryRight >= self.cannibalRight):
            return True
        return False

    def __eq__(self, other):
        return self.cannibalLeft == other.cannibalLeft and self.missionaryLeft == other.missionaryLeft \
               and self.boat == other.boat and self.cannibalRight == other.cannibalRight \
               and self.missionaryRight == other.missionaryRight

    def __hash__(self):
        return hash((self.cannibalLeft, self.missionaryLeft, self.boat, self.cannibalRight, self.missionaryRight))

def successors(state):
    children = []
    if state.boat == 'left':
        new_states = [
            State(state.cannibalLeft, state.missionaryLeft - 2, 'right',
                  state.cannibalRight, state.missionaryRight + 2),
            State(state.cannibalLeft - 2, state.missionaryLeft, 'right',
                  state.cannibalRight + 2, state.missionaryRight),
            State(state.cannibalLeft - 1, state.missionaryLeft - 1, 'right',
                  state.cannibalRight + 1, state.missionaryRight + 1),
            State(state.cannibalLeft, state.missionaryLeft - 1, 'right',
                  state.cannibalRight, state.missionaryRight + 1),
            State(state.cannibalLeft - 1, state.missionaryLeft, 'right',
                  state.cannibalRight + 1, state.missionaryRight)
        ]
    else:
        new_states = [
            State(state.cannibalLeft, state.missionaryLeft + 2, 'left',
                  state.cannibalRight, state.missionaryRight - 2),
            State(state.cannibalLeft + 2, state.missionaryLeft, 'left',
                  state.cannibalRight - 2, state.missionaryRight),
            State(state.cannibalLeft + 1, state.missionaryLeft + 1, 'left',
                  state.cannibalRight - 1, state.missionaryRight - 1),
            State(state.cannibalLeft, state.missionaryLeft + 1, 'left',
                  state.cannibalRight, state.missionaryRight - 1),
            State(state.cannibalLeft + 1, state.missionaryLeft, 'left',
                  state.cannibalRight - 1, state.missionaryRight)
        ]
    
    for new_state in new_states:
        if new_state.is_valid():
            new_state.parent = state
            children.append(new_state)
    return children

def print_solution(solution):
    if solution is None:
        print("No solution exists.")
        return
    if isinstance(solution, str) and solution == 'cutoff':
        print("Solution not found within depth limit.")
        return
    if not isinstance(solution, State):
        print(f"Unexpected solution type: {type(solution)}")
        return
    
    path = []
    path.append(solution)
    parent = solution.parent
    while parent:
        path.append(parent)
        parent = parent.parent
    for t in range(len(path)):
        state = path[len(path) - t - 1]
        print(f"({state.cannibalLeft},{state.missionaryLeft},{state.boat},{state.cannibalRight},{state.missionaryRight})")

def breadth_first_search():
    print("=== BFS ===")
    initial_state = State(3,3,'left',0,0)
    if initial_state.is_goal():
        return initial_state
    frontier = list()
    explored = set()
    frontier.append(initial_state)
    while frontier:
        state = frontier.pop(0)
        if state.is_goal():
            return state
        explored.add(state)
        children = successors(state)
        for child in children:
            if (child not in explored) and (child not in frontier):
                frontier.append(child)
    return None

def depth_first_search():
    print("=== DFS ===")
    initial_state = State(3,3,'left',0,0)
    if initial_state.is_goal():
        return initial_state
    frontier = list()
    explored = set()
    frontier.insert(0, initial_state)
    while frontier:
        state = frontier.pop(0)
        if state.is_goal():
            return state
        explored.add(state)
        children = successors(state)
        for child in children:
            if (child not in explored) and (child not in frontier):
                frontier.insert(0,child)
    return None

def depth_limited_search(depth_limit):
    print(f"=== DLS (Depth Limit: {depth_limit}) ===")
    def recursive_dls(state, depth):
        if state.is_goal():
            return state
        elif depth == 0:
            return 'cutoff'
        else:
            cutoff_occurred = False
            for child in successors(state):
                result = recursive_dls(child, depth - 1)
                if result == 'cutoff':
                    cutoff_occurred = True
                elif result is not None:
                    return result
            return 'cutoff' if cutoff_occurred else None

    return recursive_dls(State(3,3,'left',0,0), depth_limit)

def main():
    print("Missionaries and Cannibals solution:")
    print("(cannibalLeft,missionaryLeft,boat,cannibalRight,missionaryRight)")
    
    bfs_solution = breadth_first_search()
    print("\nBreadth-First Search Solution:")
    print_solution(bfs_solution)
    
    dfs_solution = depth_first_search()
    print("\nDepth-First Search Solution:")
    print_solution(dfs_solution)
    
    depth_limit = 10  # You can adjust this value
    dls_solution = depth_limited_search(depth_limit)
    print("\nDepth-Limited Search Solution:")
    print_solution(dls_solution)

if __name__ == "__main__":
    main()

Missionaries and Cannibals solution:
(cannibalLeft,missionaryLeft,boat,cannibalRight,missionaryRight)
=== BFS ===

Breadth-First Search Solution:
(3,3,left,0,0)
(1,3,right,2,0)
(2,3,left,1,0)
(0,3,right,3,0)
(1,3,left,2,0)
(1,1,right,2,2)
(2,2,left,1,1)
(2,0,right,1,3)
(3,0,left,0,3)
(1,0,right,2,3)
(1,1,left,2,2)
(0,0,right,3,3)
=== DFS ===

Depth-First Search Solution:
(3,3,left,0,0)
(2,2,right,1,1)
(2,3,left,1,0)
(0,3,right,3,0)
(1,3,left,2,0)
(1,1,right,2,2)
(2,2,left,1,1)
(2,0,right,1,3)
(3,0,left,0,3)
(1,0,right,2,3)
(2,0,left,1,3)
(0,0,right,3,3)
=== DLS (Depth Limit: 10) ===

Depth-Limited Search Solution:
Solution not found within depth limit.
